In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

from model import SiameseNet
from hybrid import HybridNet

from data_utils import QuoraDataset, DataIterator, DataIteratorAE
from config import Config
from embeddings import load_embeddings

/home/bruno/Apps/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
task = "inference"
# task = "autoencoder"
# task = "joint"
restrict = 25000

In [4]:
### Loading config and pretrained Glove embeddings
config = Config()
config.log_path += "{}_{}/".format(task, restrict)
loaded_embeddings, (w2idx, idx2w) = load_embeddings(config.glove_filename, binary=False)

Loading from saved word_embeddings
Loading vocab


In [5]:
### Loading Quora Datasets
qd_train = QuoraDataset(config.train_filename, save_path=config.train_save)
w2idx_train, idx2w_train = qd_train.w2idx, qd_train.idx2w

embeddings = np.random.normal(scale=0.001, size=(len(w2idx_train), config.we_dim))

In [6]:
for w, i in w2idx_train.items():
    idx = w2idx.get(w)
    if idx is not None:
        embeddings[i] = loaded_embeddings[idx]

In [7]:
qd_dev  = QuoraDataset(config.dev_filename, w2idx=w2idx_train, save_path=config.dev_save)
qd_test = QuoraDataset(config.test_filename, w2idx=w2idx_train, save_path=config.test_save)

In [8]:
train_data = qd_train.data(padlen=config.padlen)
dev_data = qd_dev.data(padlen=config.padlen)
test_data = qd_test.data(padlen=config.padlen)

In [ ]:
### HybridNet
model = HybridNet(config, embeddings)
model.build()

In [ ]:
model.train(train_data, dev_data, test_data, restrict=restrict, task=task)

  0%|          | 0/1562 [00:00<?, ?it/s]

Training in hid-128_feats-dist_lr-adam-0.001-relu_bs-16_drop-0.0_bn-1_emb-0_padlen-40/
Epoch 1/10 :
Supervised training of inference


 64%|██████▍   | 1000/1562 [01:42<01:01,  9.17it/s]

Step 1000/1562
dev acc inf : 75.05


 64%|██████▍   | 1003/1562 [02:02<38:56,  4.18s/it]

test acc inf : 74.77


100%|█████████▉| 1560/1562 [02:58<00:00, 10.01it/s]

Step 1561/1562
dev acc inf : 76.20


100%|██████████| 1562/1562 [03:19<00:00,  3.29s/it]

test acc inf : 75.68
